In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Dataframe-vs-SQL").getOrCreate()

24/08/12 21:48:49 WARN Utils: Your hostname, abdul-basit-HP-ProBook-450-G8-Notebook-PC resolves to a loopback address: 127.0.1.1; using 10.11.16.43 instead (on interface wlp0s20f3)
24/08/12 21:48:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/12 21:48:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark_df = spark.read.csv("Titanic.csv", inferSchema=True, header=True)
spark_df.show(2)

+-----------+------+------------+----+---+-----+-----+--------------------+------+--------+--------+
|PassengerId|Pclass|        Name| Sex|Age|SibSp|Parch|              Ticket|  Fare|Embarked|Survived|
+-----------+------+------------+----+---+-----+-----+--------------------+------+--------+--------+
|          1|     3|Allison Hill|male| 17|    4|    2|43d75413-a939-4bd...|144.08|       Q|       1|
|          2|     1| Noah Rhodes|male| 60|    2|    2|6334fa2a-8b4b-47e...|249.04|       S|       0|
+-----------+------+------------+----+---+-----+-----+--------------------+------+--------+--------+
only showing top 2 rows



In [20]:
spark_df.createOrReplaceTempView("titanic")
sql_df = spark.sql("select * from titanic")
sql_df.show(2)


+-----------+------+------------+----+---+-----+-----+--------------------+------+--------+--------+
|PassengerId|Pclass|        Name| Sex|Age|SibSp|Parch|              Ticket|  Fare|Embarked|Survived|
+-----------+------+------------+----+---+-----+-----+--------------------+------+--------+--------+
|          1|     3|Allison Hill|male| 17|    4|    2|43d75413-a939-4bd...|144.08|       Q|       1|
|          2|     1| Noah Rhodes|male| 60|    2|    2|6334fa2a-8b4b-47e...|249.04|       S|       0|
+-----------+------+------------+----+---+-----+-----+--------------------+------+--------+--------+
only showing top 2 rows



In [45]:
from pyspark.sql import functions as F

spark_df_grpBy = spark_df.groupBy("Pclass").count()

In [46]:
sql_df_grpBy = spark.sql("""
                        SELECT Pclass, COUNT(*) as count
                         FROM titanic
                         GROUP BY Pclass
                         """)

In [50]:
spark_df_grpBy.explain()
sql_df_grpBy.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Pclass#18], functions=[count(1)])
   +- Exchange hashpartitioning(Pclass#18, 200), ENSURE_REQUIREMENTS, [plan_id=192]
      +- HashAggregate(keys=[Pclass#18], functions=[partial_count(1)])
         +- FileScan csv [Pclass#18] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/abdul-basit/Documents/Data_Engineering/Spark/Titanic.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Pclass:int>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Pclass#18], functions=[count(1)])
   +- Exchange hashpartitioning(Pclass#18, 200), ENSURE_REQUIREMENTS, [plan_id=205]
      +- HashAggregate(keys=[Pclass#18], functions=[partial_count(1)])
         +- FileScan csv [Pclass#18] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/abdul-basit/Documents/Data_Engineering/Spark/Titanic.csv], PartitionF

In [70]:
from pyspark.sql.functions import max
max_fare = spark_df.agg(max("Fare").alias("max_fare")).collect()[0]["max_fare"]

# Filter the original DataFrame to get the row(s) with the maximum fare
max_df = spark_df.filter(spark_df["Fare"] == max_fare).select("Name", "Fare")
max_df.show()


max_df_sql = spark.sql("""
SELECT Name, Fare
FROM titanic
WHERE Fare = (SELECT MAX(Fare) FROM titanic)
""")

# max_df.show()
max_df_sql.show()



+-----------------+------+
|             Name|  Fare|
+-----------------+------+
|Christina Walters|499.78|
+-----------------+------+

+-----------------+------+
|             Name|  Fare|
+-----------------+------+
|Christina Walters|499.78|
+-----------------+------+

